In [1]:
import argparse
import os
import numpy as np
import torch
import cv2, glob
os.environ["CUDA_VISIBLE_DEVICES"] = "1"
from modeling.sync_batchnorm.replicate import patch_replication_callback
from modeling.deeplab import *

from dataloaders.utils import decode_segmap
MI = np.load('meanimage.npy')

numClass={'pascal':3,
'coco':21,
'cityscapes':19}
classes = ['scar', 'others']
for cid, c in enumerate(classes):
    if not os.path.isdir('result/'+c):
        os.mkdir('result/' + c)

# parser = argparse.ArgumentParser(description="PyTorch DeeplabV3Plus Training")

# parser.add_argument('--backbone', type=str, default='resnet',choices=['resnet', 'xception', 'drn', 'mobilenet'],help='backbone name (default: resnet)')
# parser.add_argument('--out-stride', type=int, default=16,help='network output stride (default: 8)')
# parser.add_argument('--dataset', type=str, default='cityscapes', choices=['pascal', 'coco', 'cityscapes'],help='dataset name (default: pascal)')
# parser.add_argument('--sync-bn', type=bool, default=None,help='whether to use sync bn (default: auto)')
# parser.add_argument('--freeze-bn', type=bool, default=False,help='whether to freeze bn parameters (default: False)')
# parser.add_argument('--weightPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--imgPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--outPath', type=str, default=None,help='put the path to resuming file if needed')
# parser.add_argument('--no-cuda', action='store_true', default=False, help='disables CUDA training')
# args = parser.parse_args()
cuda = torch.cuda.is_available()
cuda = True
loc = 'cuda' if cuda else 'cpu'
nclass = numClass['pascal']
model = DeepLab(num_classes=nclass, backbone='resnet', output_stride=8, sync_bn=None, freeze_bn=False)
weight_dict=torch.load('run/pascal/task2class2/model_best.pth.tar', map_location=loc)
if cuda:
    model = torch.nn.DataParallel(model, device_ids=[0])
    patch_replication_callback(model)
    model = model.cuda()
    
    model.module.load_state_dict(weight_dict['state_dict'])
else:
    model.load_state_dict(weight_dict['state_dict'])
model.eval()


filenames = glob.glob('Validation-400-images/*.jpg')
kernel = np.ones((5,5),np.uint8)
kernel2 = np.ones((11, 11),np.uint8)
for imgPath in filenames:
    fn = os.path.basename(imgPath)[:-4]
    outPath = 'result/' + fn +'.png'

    image = cv2.imread(imgPath)
    oriDim = image.shape
    image = cv2.resize(image, dsize=(480,480)) - cv2.resize(MI, (480,480))
#     image = cv2.resize(image, dsize=(513,513)) 
    image = image.astype(np.float32) / 255.
    image = image[:, :, ::-1]
    means = [0.485, 0.456, 0.406]
    stds = [0.229, 0.224, 0.225]
    
    
    
    for i in range(3):
        image[:, :, i] = image[:, :, i] - means[i]
        image[:, :, i] = image[:, :, i] / stds[i]

    image = torch.from_numpy(image.transpose((2, 0, 1)).astype(np.float32)).float().unsqueeze(0)

    if cuda:
        image = image.cuda()
        
    with torch.no_grad():
        output = model(image)
        output = output.data.cpu().numpy()
        prediction = np.argmax(output, axis=1)[0]
        ps=[]
        for cid, c in enumerate(classes):
            mask = np.zeros((prediction.shape[0], prediction.shape[1]), np.uint8) +255
            mask[prediction == cid+1] = 0
#             mask = output[0, cid+1]
#             mask[mask>0.3] = 255
#             mask[mask<=0.3] = 0
            mask = mask.astype(np.uint8)
            mask = cv2.morphologyEx(255-mask, cv2.MORPH_OPEN, kernel)
            mask = cv2.morphologyEx(mask, cv2.MORPH_DILATE, kernel2)
            hole=mask.copy()

            cv2.floodFill(hole,None,(0,0),255) # 找到洞孔
            hole=cv2.bitwise_not(hole)
            mask=255-cv2.bitwise_or(mask,hole)

            mask = cv2.resize(mask,dsize=(oriDim[1],oriDim[0]), interpolation=cv2.INTER_NEAREST)
            ps.append(np.mean(prediction == cid+1))
            cv2.imwrite('result/' + c + '/' + fn+'.png', mask)

        segmap = decode_segmap(prediction, dataset='pascal')
        segmap = (segmap*255).astype(np.uint8)
        segmap = cv2.resize(segmap,dsize=(oriDim[1],oriDim[0]))
        segmap = segmap[:, :, ::-1]
        cv2.imwrite(outPath, segmap)
    print('Done inference '+fn,'percentage:', ps)
exit(1)

Done inference V0366 percentage: [0.0, 0.0]
Done inference V0213 percentage: [0.0, 0.0]
Done inference V0333 percentage: [0.0, 0.0]
Done inference V0108 percentage: [0.0, 0.0]
Done inference V0264 percentage: [0.0, 0.0]
Done inference V0323 percentage: [0.04966145833333333, 0.0]
Done inference V0347 percentage: [0.0, 0.0]
Done inference V0064 percentage: [0.0, 0.0]
Done inference V0389 percentage: [0.002534722222222222, 0.0]
Done inference V0381 percentage: [0.0, 0.0]
Done inference V0306 percentage: [0.0, 0.0]
Done inference V0307 percentage: [0.0, 0.0]
Done inference V0188 percentage: [0.0042664930555555555, 0.0]
Done inference V0014 percentage: [0.0, 0.0]
Done inference V0296 percentage: [0.0, 0.0]
Done inference V0282 percentage: [0.0, 0.0]
Done inference V0325 percentage: [0.0, 0.0]
Done inference V0057 percentage: [0.0, 0.0]
Done inference V0149 percentage: [0.0, 0.0]
Done inference V0348 percentage: [0.0, 0.0]
Done inference V0055 percentage: [0.0, 0.0]
Done inference V0243 perc

Done inference V0398 percentage: [0.0, 0.0]
Done inference V0169 percentage: [0.0, 0.0]
Done inference V0062 percentage: [0.0, 0.0]
Done inference V0267 percentage: [0.0, 0.0]
Done inference V0085 percentage: [0.08598958333333333, 0.0]
Done inference V0203 percentage: [0.0, 0.0]
Done inference V0276 percentage: [0.0, 0.0]
Done inference V0328 percentage: [0.0, 0.0]
Done inference V0360 percentage: [0.0, 0.0]
Done inference V0222 percentage: [0.0, 0.0]
Done inference V0033 percentage: [0.0, 0.0]
Done inference V0097 percentage: [0.0, 0.0]
Done inference V0164 percentage: [0.0, 0.0]
Done inference V0096 percentage: [0.0, 0.0]
Done inference V0377 percentage: [0.0, 0.0]
Done inference V0092 percentage: [0.0, 0.0]
Done inference V0089 percentage: [0.0, 0.0]
Done inference V0367 percentage: [0.0, 0.0]
Done inference V0046 percentage: [0.0, 0.0]
Done inference V0182 percentage: [0.0, 0.0]
Done inference V0122 percentage: [0.0, 0.0]
Done inference V0309 percentage: [0.0, 0.0]
Done inference V

Done inference V0117 percentage: [0.0, 0.0]
Done inference V0209 percentage: [0.0, 0.0]
Done inference V0031 percentage: [0.0, 0.0]
Done inference V0275 percentage: [0.0, 0.0]
Done inference V0313 percentage: [0.0, 0.0]
Done inference V0242 percentage: [0.0, 0.0]
Done inference V0176 percentage: [0.0, 0.0]
Done inference V0171 percentage: [0.0, 0.0]
Done inference V0246 percentage: [0.0, 0.0]
Done inference V0054 percentage: [0.0, 0.0]
Done inference V0386 percentage: [0.0, 0.0]
Done inference V0259 percentage: [0.0, 0.0]
Done inference V0368 percentage: [0.004696180555555556, 0.0]
Done inference V0310 percentage: [0.0, 0.0]
Done inference V0040 percentage: [0.0, 0.0]
Done inference V0196 percentage: [0.00888454861111111, 0.0]
Done inference V0214 percentage: [0.0, 0.0]
Done inference V0190 percentage: [0.036705729166666666, 0.0]
Done inference V0099 percentage: [0.09681857638888888, 0.0]
Done inference V0291 percentage: [0.0, 0.0]
Done inference V0270 percentage: [0.0, 0.0]
Done infer